In [1]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import dotenv
import os

In [2]:
load_dotenv()
LLAMA_3_API_KEY = os.getenv("LLAMA_3_API_KEY")

In [3]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = LLAMA_3_API_KEY,
    model_name = "llama-3.1-70b-versatile"
)

In [4]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from datetime import date

class JobDescription(BaseModel):
    role: str = Field(..., description="Job title")
    experience: float = Field(..., description="Experience level required for the job")
    acceptable_majors: List[str] = Field(..., description="Acceptable majors for the job")
    skills: List[str] = Field(..., description="Skills required for the job")

In [5]:
page_data = '''
Bằng Đại học / Thạc sĩ chuyên ngành Khoa học máy tính, Thống kê, Khoa học dữ liệu, AI, Học máy hoặc các lĩnh vực liên quan.
Có ít nhất 4 - 5 năm kinh nghiệm làm việc trong lĩnh vực AI.
Kinh nghiệm quản lý nhóm, đã từng lãnh đạo một nhóm gồm ít nhất 3 kỹ sư AI.
Kinh nghiệm trong việc phát triển giải pháp AI, đặc biệt là ứng dụng Generative AI.
Chuyên sâu về NLP, Computer Vision, Recommendation Systems.
Kinh nghiệm tích hợp AI vào sản phẩm và dịch vụ, đảm bảo đáp ứng được nhu cầu kinh doanh.
Kinh nghiệm xây dựng và mở rộng các sản phẩm / dịch vụ AI trong môi trường Production.
Kinh nghiệm với các framework ML hiện đại (PyTorch, TensorFlow, Hugging Face...)
Tư duy và tính sáng tạo để hiểu phát triển mô hình AI theo nhu cầu của tổ chức.
Tiếng Anh thành thạo, sẵn sàng làm việc với đối tác.
'''

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=JobDescription)

prompt = PromptTemplate(
    template="""        
          ### SCRAPED TEXT FROM WEBSITE:
          {page_data}
          ### INSTRUCTION:
          The scraped text is from the career's page of a website.
          Your job is to extract the job postings and return them in JSON format.
          Only return the valid JSON.
          ### VALID JSON (NO PREAMBLE): 
          \n{format_instructions}\n
     """,
     input_variables=["query"],
     partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [7]:
res = chain.invoke(input={'page_data':page_data})
res

{'role': 'Trưởng nhóm AI',
 'experience': 4,
 'acceptable_majors': ['Khoa học máy tính',
  'Thống kê',
  'Khoa học dữ liệu',
  'AI',
  'Học máy'],
 'skills': ['Quản lý nhóm',
  'Phát triển giải pháp AI',
  'Generative AI',
  'NLP',
  'Computer Vision',
  'Recommendation Systems',
  'Tích hợp AI vào sản phẩm và dịch vụ',
  'Xây dựng và mở rộng các sản phẩm / dịch vụ AI',
  'PyTorch',
  'TensorFlow',
  'Hugging Face',
  'Tiếng Anh']}